In [5]:
import numpy as np 
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [6]:
df = pd.read_csv("Q1_clean.csv")
data = df.fillna(0)

In [7]:
avg = data.groupby('Neighbourhood ')[['Price', 'Review Scores Rating']].mean()

In [8]:
dummy = pd.get_dummies(data['Neighbourhood '],data['Review Scores Rating'], data['Property Type'], drop_first=True)
encode = pd.concat([data, dummy], axis = 1)
y = encode['Price']
X_train, X_test, y_train, y_test = train_test_split(dummy, y, test_size = 0.2,random_state=5)